In [1]:
import pickle
import numpy as np
import gc

In [2]:
'''读原始test文件'''

test_filename = "../data/raw_test.pkl"
with open(test_filename, "rb") as f:
    data = pickle.load(f)
    
test_w = data['test_w']
test_r = data['test_r']
print(len(test_w), len(test_w)==len(test_r))

del data
gc.collect()

1739 True


0

In [3]:
FORMER_E_CNT_THRESHOLD = 5
LATTER_E_CNT_THRESHOLD = 2
MAX_AVAILABLE_ACTION = 83
MAX_AWARD = 4140.0
MIN_AWARD = 0.0
MAX_INDUSTRY_VALUE = 36
MAX_CATEGORY_VALUE = 6
MAX_PROJECT_QUALITY = 10.797708090145798
MIN_PROJECT_QUALITY = 0.0
MAX_START_GAP = 381
MIN_START_GAP = 0
MAX_END_GAP = 338
MIN_END_GAP = 0

In [4]:
'''将原始test经归一化和填补0后转为numpy格式'''

batch_states, batch_actions, batch_rewards_w, batch_rewards_r, batch_infos = [], [], [], [], []
for i in range(len(test_w)):
    # 【S】
    state_list = []
    for row_state in test_w[i]['state']:
        row_state_list = []
        row_state_list.append(row_state['a_category'])
        for j in range(1, FORMER_E_CNT_THRESHOLD+1):
            row_state_list.append(row_state['l'+str(j)+'_category'])

        row_state_list.append(row_state['a_industry'])
        for j in range(1, FORMER_E_CNT_THRESHOLD+1):
            row_state_list.append(row_state['l'+str(j)+'_industry'])

        row_state_list.append((row_state['a_award']-MIN_AWARD)/(MAX_AWARD-MIN_AWARD))
        for j in range(1, FORMER_E_CNT_THRESHOLD+1):
            row_state_list.append((row_state['l'+str(j)+'_award']-MIN_AWARD)/(MAX_AWARD-MIN_AWARD))

        row_state_list.append((row_state['a_start_gap']-MIN_START_GAP)/(MAX_START_GAP-MIN_START_GAP))
        start_gap_sum = 0
        for j in range(1, FORMER_E_CNT_THRESHOLD+1):
            start_gap_sum += (row_state['l'+str(j)+'_start_gap']-MIN_START_GAP)/(MAX_START_GAP-MIN_START_GAP)
        row_state_list.append(start_gap_sum/FORMER_E_CNT_THRESHOLD)

        row_state_list.append((row_state['a_end_gap']-MIN_END_GAP)/(MAX_END_GAP-MIN_END_GAP))
        end_gap_sum = 0
        for j in range(1, FORMER_E_CNT_THRESHOLD+1):
            end_gap_sum += (row_state['l'+str(j)+'_end_gap']-MIN_END_GAP)/(MAX_END_GAP-MIN_END_GAP)
        row_state_list.append(end_gap_sum/FORMER_E_CNT_THRESHOLD)
        
        row_state_list.append(row_state['worker_quality'])

        row_state_list.append((row_state['project_quality']-MIN_PROJECT_QUALITY)/(MAX_PROJECT_QUALITY-MIN_PROJECT_QUALITY))
        state_list.append(row_state_list)
    np_state = np.array(state_list)
    np_state = np.concatenate((np_state, np.zeros([MAX_AVAILABLE_ACTION - np_state.shape[0], np_state.shape[1]])), axis=0)
    batch_states.append(np_state)
    # 【A】
    batch_actions.append(test_w[i]['action'])
    # 【R】
    batch_rewards_w.append(test_w[i]['reward'])
    batch_rewards_r.append(test_r[i]['reward'])
    # 【INFO】
    info_list = []
    for row_state in test_w[i]['state']:
        info_list.append([len(test_w[i]['state']),  # legal_max_action
                          row_state['a_category'],
                          row_state['a_industry'],
                          row_state['a_award'],
                          row_state['a_start_gap'],
                          row_state['a_end_gap'],
                          row_state['worker_quality'],
                          row_state['project_quality']])
    np_info = np.array(info_list)
    np_info = np.concatenate((np_info, np.zeros([MAX_AVAILABLE_ACTION - np_info.shape[0], np_info.shape[1]])), axis=0)
    batch_infos.append(np_info)

print(np.array(batch_states).shape)
print(np.array(batch_actions).shape)
print(np.array(batch_rewards_w).shape)
print(np.array(batch_rewards_r).shape)
print(np.array(batch_infos).shape)
assert len(batch_states) == len(batch_actions) == len(batch_rewards_w) == len(batch_rewards_r) == len(batch_infos)

test_transition_filename = "../exp_data/test_transition.pkl"
with open(test_transition_filename, "wb") as f:
    pickle.dump({'states': np.array(batch_states),
                 'actions': np.array(batch_actions, dtype=np.int32),
                 'rewards_w': np.array(batch_rewards_w),
                 'rewards_r': np.array(batch_rewards_r),
                 'infos': np.array(batch_infos)
                 }, f)

(1739, 83, 24)
(1739,)
(1739,)
(1739,)
(1739, 83, 8)


In [16]:
test_transition_filename = "../exp_data/test_transition.pkl"
with open(test_transition_filename, "rb") as f:
    data = pickle.load(f)
infos = data['infos']
len(infos)

1739

1739

In [15]:
np.mean(infos[:, 0, 0])

47.439332949971245

In [14]:
np.sum(infos[:, 0, 0] == 1)

6